# improved feature engineering

## importing the datasets needed for this analysis

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from tqdm import tqdm
from functools import reduce
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, StackingRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.model_selection import train_test_split
from catboost import Pool, cv
from collections import Counter
from functools import reduce
from tqdm import tqdm
from itertools import cycle
from scipy import stats
from scipy.stats import skew, kurtosis
from sklearn.decomposition import PCA
import seaborn as sns

In [ ]:
train_logs=pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv')
train_scores=pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv')
test_logs=pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv')

In [ ]:
train_logs.shape

My focus will be on the train logs dataset. It carries a lot of features that I think are important for this analyis.  but lets look at the train_scores first. 

In [ ]:
train_scores. describe()

We can see that the highest score was 6 and the lowest score was 0.500 with the average score being  3.7. Lets plot this in a histogram and see the results. 

In [ ]:
train_scores.score.hist()

We look at train_logs dataset and evaluate it. 

In [ ]:
train_logs.tail(20)

In [ ]:
train_logs[train_logs['id']=='001519c8']

In [ ]:
print(train_logs.down_event.value_counts())

In [ ]:
train_logs.up_event.value_counts()[:100]

In [ ]:
train_logs.text_change.value_counts()


In [ ]:
train_logs.id.value_counts()

In [ ]:
train_logs.cursor_position.describe().round(2)

In [ ]:
train_logs.event_id.describe().round(2)

In [ ]:
train_logs.event_id.hist()

In [ ]:
train_logs['activity'].value_counts()

# Feature engineering

In [ ]:
def get_flights(df):
    return df['down_time'].shift(-1) - df['up_time']

def flight_time(data):
    means = []
    stds = []
    mins = []
    maxs = []
    medians = []

    grouped_data = data.groupby('id')

    for _, df in tqdm(grouped_data):
        flight_times = get_flights(df)
        mean_time = np.mean(flight_times)
        std_time = np.std(flight_times)
        min_time = np.min(flight_times)
        max_time = np.max(flight_times)
        median_time = np.median(flight_times)

        means.append(mean_time)
        stds.append(std_time)
        mins.append(min_time)
        maxs.append(max_time)
       

    DF = pd.DataFrame({
        'id': grouped_data.groups.keys(),
        "mean flight time": means,
        'std flight time': stds,
        'minimum flight time': mins,
        'maximum flight time': maxs
    })

    return DF

In [ ]:
df=flight_time(train_logs)

In [ ]:
df.head()

In [ ]:
#get the total time spent on a the essay
def time_spent(data):
    data=data[['id', 'action_time']]
    mean_time=data.groupby('id').mean()
    return mean_time
    

In [ ]:
times=time_spent(train_logs)
times.reset_index(inplace=True)
times



In [ ]:
actions = pd.DataFrame({
    'actions': train_logs['id'].value_counts()
})



In [ ]:
actions.reset_index(inplace=True)
actions



In [ ]:
# pause measures
def pauses(data):

    Nonproduction=data[data['activity']=='Nonproduction']
    No_pauses=Nonproduction.id.value_counts()
    return No_pauses
pauses_count=pauses(train_logs)

In [ ]:
pauses_count

In [ ]:
pauses_count = pd.DataFrame({
    'pauses': pauses(train_logs)
})
pauses_count.reset_index(inplace=True)
pauses_count



In [ ]:
# backspace count
def backspace_count(data):
    Backspace=data[data['up_event']=='Backspace']
    return Backspace.id.value_counts()
backspace=backspace_count(train_logs)

backspace = pd.DataFrame({
    'backspace': backspace_count(train_logs)
})
backspace.reset_index(inplace=True)



In [ ]:
backspace

In [ ]:
backspace

In [ ]:
# words per minute- I omit the  non production
def words_per_minute(data):
    production=data[data['activity']!='Nonproduction']
    # getting total minutes in active production
    production_active=production[['id', 'action_time']]
    total_minutes=production_active.groupby('id').sum()['action_time']/60000
    # getting the total words
    words_df=production[['id', 'word_count']]
    words=words_df.groupby('id').max()['word_count']
    return words/total_minutes
    

In [ ]:
def keystrokes_per_minute(data):
    production=data[data['activity']=='Input']
    production_active=production[['id', 'action_time']]
    total_minutes=production_active.groupby('id').sum()['action_time']/60000
    strokes_df=production[['id', 'down_event']]
    strokes=strokes_df.groupby('id').count()['down_event']
    return strokes/total_minutes

keystrokes = pd.DataFrame({
    'keystrokes_per_minute': keystrokes_per_minute(train_logs)
})
keystrokes.reset_index(inplace=True)


In [ ]:
def pastes(data):
    pastes=data[data['activity']=='Paste']
    df=pastes[['id', 'activity']]
    return df.groupby('id').count()['activity']
paste_text= pd.DataFrame({
    'paste_text': pastes(train_logs)
})
paste_text.reset_index(inplace=True)
keystrokes = pd.DataFrame({
    'keystrokes_per_minute': keystrokes_per_minute(train_logs)
})
keystrokes.reset_index(inplace=True)
typing_speed = pd.DataFrame({
    'typing_speed': words_per_minute(train_logs)
})
typing_speed.reset_index(inplace=True)

backspace=backspace_count(train_logs)

backspace = pd.DataFrame({
    'backspace': backspace_count(train_logs)
})
backspace.reset_index(inplace=True)

pauses_count = pd.DataFrame({
    'pauses': pauses(train_logs)
})
pauses_count.reset_index(inplace=True)



times=time_spent(train_logs)
times.reset_index(inplace=True)

df=flight_time(train_logs)
actions = pd.DataFrame({
    'actions': train_logs['id'].value_counts()
})
actions.reset_index(inplace=True)


In [ ]:
def cut_text(data):
    cuts=data[data['activity']=='Remove/Cut']
    df=cuts[['id', 'activity']]
    return df.groupby('id').count()['activity']
cut_text(train_logs)

cuts= pd.DataFrame({
    'cut_text': cut_text(train_logs)
})
cuts.reset_index(inplace=True)
paste_text= pd.DataFrame({
    'paste_text': pastes(train_logs)
})
paste_text.reset_index(inplace=True)
keystrokes = pd.DataFrame({
    'keystrokes_per_minute': keystrokes_per_minute(train_logs)
})
keystrokes.reset_index(inplace=True)
typing_speed = pd.DataFrame({
    'typing_speed': words_per_minute(train_logs)
})
typing_speed.reset_index(inplace=True)

backspace=backspace_count(train_logs)

backspace = pd.DataFrame({
    'backspace': backspace_count(train_logs)
})
backspace.reset_index(inplace=True)

pauses_count = pd.DataFrame({
    'pauses': pauses(train_logs)
})
pauses_count.reset_index(inplace=True)



times=time_spent(train_logs)
times.reset_index(inplace=True)

df=flight_time(train_logs)
actions = pd.DataFrame({
    'actions': train_logs['id'].value_counts()
})
actions.reset_index(inplace=True)


In [ ]:
def replace_text(data):
    replaces=data[data['activity']=='Replace']
    df=replaces[['id', 'activity']]
    return df.groupby('id').count()['activity']
replace_text(train_logs)






In [ ]:
# get punctuations
import string

def is_punctuation_only(cell_value):
    return all(char in string.punctuation for char in cell_value)

train_logs['is_punctuation_only'] = train_logs['text_change'].apply(is_punctuation_only)

punctuation_counts = train_logs.groupby('id')['is_punctuation_only'].sum().reset_index()


In [ ]:
punctuation_counts

In [ ]:
# input counts
def input_counts(data):
    data = data[['id', 'activity','text_change']]
    input_occurrences = data[data['activity'] == 'Input'].groupby('id')['text_change'].count().reset_index()
    return input_occurrences

In [ ]:
input_counts(train_logs)

In [ ]:

inputs= input_counts(train_logs)




In [ ]:
# determine the characters that were replaced
# first lets view the data
train_logs[train_logs['activity']=='Replace']

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def change_count(data):
    replace_data = data[data['activity'] == 'Replace']
    splits = replace_data['text_change'].str.split('=>')
    
    replaced = splits.apply(lambda x: len(x[0]))
    changed = splits.apply(lambda x: len(x[1]))

    sum_replaced = replaced.sum()
    sum_changed = changed.sum()
    average_replaced = replaced.mean()
    average_changed = changed.mean()

    return sum_replaced, sum_changed, average_replaced, average_changed

def counts_of_changes(data):
    sum_replace = []
    sum_change = []
    average_replace = []

    for i in tqdm(data['id'].unique(), desc='Processing IDs'):
        new_data = data[data['id'] == i]
        (
            sum_replaced,
            sum_changed,
            average_replaced,
            average_changed
        ) = change_count(new_data)

        sum_replace.append(sum_replaced)
        sum_change.append(sum_changed)
        average_replace.append(average_replaced)

    result_df = pd.DataFrame({
        'id': data['id'].unique(),
        'sum_replace': sum_replace,
        'sum_change': sum_change,
        'average_replace': average_replace,
        'average_changed': average_changed
    })

    return result_df




In [ ]:
class Preprocessor:
    
    def __init__(self, seed):
        self.seed = seed
        
        self.activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
        self.events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 
              'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
        self.text_changes = ['q', ' ', 'NoChange', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']
        self.punctuations = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/',
                        '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']
        self.gaps = [1, 2, 3, 5, 10, 20, 50, 100]
#         self.gaps = [1, 2]
    
    def activity_counts(self, df):
        tmp_df = df.groupby('id').agg({'activity': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['activity'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.activities:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'activity_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols
        return ret


    def event_counts(self, df, colname):
        tmp_df = df.groupby('id').agg({colname: list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df[colname].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.events:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'{colname}_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols
        return ret


    def text_change_counts(self, df):
        tmp_df = df.groupby('id').agg({'text_change': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['text_change'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.text_changes:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'text_change_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols
        return ret

    def match_punctuations(self, df):
        tmp_df = df.groupby('id').agg({'down_event': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['down_event'].values):
            cnt = 0
            items = list(Counter(li).items())
            for item in items:
                k, v = item[0], item[1]
                if k in self.punctuations:
                    cnt += v
            ret.append(cnt)
        ret = pd.DataFrame({'punct_cnt': ret})
        return ret


    def get_input_words(self, df):
        tmp_df = df[(~df['text_change'].str.contains('=>'))&(df['text_change'] != 'NoChange')].reset_index(drop=True)
        tmp_df = tmp_df.groupby('id').agg({'text_change': list}).reset_index()
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: ''.join(x))
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: re.findall(r'q+', x))
        tmp_df['input_word_count'] = tmp_df['text_change'].apply(len)
        tmp_df['input_word_length_mean'] = tmp_df['text_change'].apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_max'] = tmp_df['text_change'].apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_std'] = tmp_df['text_change'].apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df.drop(['text_change'], axis=1, inplace=True)
        return tmp_df
    
    def make_feats(self, df):
        
        print("Starting to engineer features")
        
        # initialize features dataframe
        feats = pd.DataFrame({'id': df['id'].unique().tolist()})
        
        # get shifted features
        # time shift
        print("Engineering time data")
        for gap in self.gaps:
            print(f"> for gap {gap}")
            df[f'up_time_shift{gap}'] = df.groupby('id')['up_time'].shift(gap)
            df[f'action_time_gap{gap}'] = df['down_time'] - df[f'up_time_shift{gap}']
        df.drop(columns=[f'up_time_shift{gap}' for gap in self.gaps], inplace=True)

        # cursor position shift
        print("Engineering cursor position data")
        for gap in self.gaps:
            print(f"> for gap {gap}")
            df[f'cursor_position_shift{gap}'] = df.groupby('id')['cursor_position'].shift(gap)
            df[f'cursor_position_change{gap}'] = df['cursor_position'] - df[f'cursor_position_shift{gap}']
            df[f'cursor_position_abs_change{gap}'] = np.abs(df[f'cursor_position_change{gap}'])
        df.drop(columns=[f'cursor_position_shift{gap}' for gap in self.gaps], inplace=True)

        # word count shift
        print("Engineering word count data")
        for gap in self.gaps:
            print(f"> for gap {gap}")
            df[f'word_count_shift{gap}'] = df.groupby('id')['word_count'].shift(gap)
            df[f'word_count_change{gap}'] = df['word_count'] - df[f'word_count_shift{gap}']
            df[f'word_count_abs_change{gap}'] = np.abs(df[f'word_count_change{gap}'])
        df.drop(columns=[f'word_count_shift{gap}' for gap in self.gaps], inplace=True)
        
        # get aggregate statistical features
        print("Engineering statistical summaries for features")
        # [(feature name, [ stat summaries to add ])]
        feats_stat = [
            ('event_id', ['max']),
            ('up_time', ['max']),
            ('action_time', ['sum', 'max', 'mean', 'std']),
            ('activity', ['nunique']),
            ('down_event', ['nunique']),
            ('up_event', ['nunique']),
            ('text_change', ['nunique']),
            ('cursor_position', ['nunique', 'max', 'mean']),
            ('word_count', ['nunique', 'max', 'mean'])]
        for gap in self.gaps:
            feats_stat.extend([
                (f'action_time_gap{gap}', ['max', 'min', 'mean', 'std', 'sum', skew, kurtosis]),
                (f'cursor_position_change{gap}', ['max', 'mean', 'std', 'sum', skew, kurtosis]),
                (f'word_count_change{gap}', ['max', 'mean', 'std', 'sum', skew, kurtosis])
            ])
        
        pbar = tqdm(feats_stat)
        for item in pbar:
            colname, methods = item[0], item[1]
            for method in methods:
                pbar.set_postfix()
                if isinstance(method, str):
                    method_name = method
                else:
                    method_name = method.__name__
                    
                pbar.set_postfix(column=colname, method=method_name)
                tmp_df = df.groupby(['id']).agg({colname: method}).reset_index().rename(columns={colname: f'{colname}_{method_name}'})
                feats = feats.merge(tmp_df, on='id', how='left')

        # counts
        print("Engineering activity counts data")
        tmp_df = self.activity_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering event counts data")
        tmp_df = self.event_counts(df, 'down_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        tmp_df = self.event_counts(df, 'up_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering text change counts data")
        tmp_df = self.text_change_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering punctuation counts data")
        tmp_df = self.match_punctuations(df)
        feats = pd.concat([feats, tmp_df], axis=1)

        # input words
        print("Engineering input words data")
        tmp_df = self.get_input_words(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')

        # compare feats
        print("Engineering ratios data")
        feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
        feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
        feats['event_time_ratio'] = feats['event_id_max']  / feats['up_time_max']
        feats['idle_time_ratio'] = feats['action_time_gap1_sum'] / feats['up_time_max']
        
        print("Done!")
        return feats

In [ ]:
def data_func(train_logs):
    inputs= input_counts(train_logs)
    train_logs['is_punctuation_only'] = train_logs['text_change'].apply(is_punctuation_only)

    punctuation_counts = train_logs.groupby('id')['is_punctuation_only'].sum().reset_index()
    replaced= pd.DataFrame({
        'replaced_text': replace_text(train_logs)
    })
    replaced.reset_index(inplace=True)
    cuts= pd.DataFrame({
        'cut_text': cut_text(train_logs)
    })
    cuts.reset_index(inplace=True)
    paste_text= pd.DataFrame({
        'paste_text': pastes(train_logs)
    })
    paste_text.reset_index(inplace=True)
    keystrokes = pd.DataFrame({
        'keystrokes_per_minute': keystrokes_per_minute(train_logs)
    })
    keystrokes.reset_index(inplace=True)
    typing_speed = pd.DataFrame({
        'typing_speed': words_per_minute(train_logs)
    })
    typing_speed.reset_index(inplace=True)

    backspace=backspace_count(train_logs)

    backspace = pd.DataFrame({
        'backspace': backspace_count(train_logs)
    })
    backspace.reset_index(inplace=True)

    pauses_count = pd.DataFrame({
        'pauses': pauses(train_logs)
    })
    pauses_count.reset_index(inplace=True)



    times=time_spent(train_logs)
    times.reset_index(inplace=True)

    df=flight_time(train_logs)
    actions = pd.DataFrame({
        'actions': train_logs['id'].value_counts()
    })
    actions.reset_index(inplace=True)
    preprocessor = Preprocessor(seed=42)

    print("Engineering features for training data")

    train_feats = preprocessor.make_feats(train_logs)
    train_agg_fe_df = train_logs.groupby("id")[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
    train_agg_fe_df.columns = ['_'.join(x) for x in train_agg_fe_df.columns]
    train_agg_fe_df = train_agg_fe_df.add_prefix("tmp_")
    train_agg_fe_df.reset_index(inplace=True)
    df_list=[ inputs, punctuation_counts,
        replaced, cuts, paste_text, keystrokes, typing_speed,
              backspace, pauses_count, times, df, actions, train_agg_fe_df
    ]
    df_merged= reduce(lambda  left,right: pd.merge(left,right,on=['id'],
                                            how='outer'), df_list)
    return df_merged

In [ ]:
df_merged=data_func(train_logs)
df_merged=pd.merge(df_merged, train_scores, on=['id'],
                                               how='outer')
df_merged.fillna(0, inplace=True)

In [ ]:
df_merged.columns

In [ ]:
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='pearson', min_periods=1)
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out

df= df_merged.drop('id', axis=1)
df=trimm_correlated(df, 0.8)

# Data splitting

In [ ]:
X=df.drop('score', axis=1)
y=df['score']

In [ ]:
normal=Normalizer()
X_Normalized=normal.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X_Normalized,y, test_size=0.2, random_state=42)

In [ ]:
model=GradientBoostingRegressor()
model.fit(X_train, y_train)
y_preds=model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_preds))

In [ ]:
# Define the base models
base_models = [
    ('catboost', CatBoostRegressor()),
    ('cat',CatBoostRegressor()),
    ('RF',RandomForestRegressor(n_estimators=500)),
    ('xgb',XGBRegressor(n_estimators=200)),
    ('xgb2',XGBRegressor()),
    ('cat2',CatBoostRegressor())
]
# Define the final estimator
final_estimator = SVR()  
# Create the stacking regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=final_estimator
)
# Fit the stacking regressor with your training data
stacking_regressor.fit(X_train, y_train)


In [ ]:
y_pred = stacking_regressor.predict(X_test )
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
test_agg_fe_df = test_logs.groupby("id")[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
test_agg_fe_df.columns = ['_'.join(x) for x in test_agg_fe_df.columns]
test_agg_fe_df = test_agg_fe_df.add_prefix("tmp_")
test_agg_fe_df.reset_index(inplace=True)

In [ ]:
test_logs.shape

In [ ]:
test=data_func(test_logs)

In [ ]:

test.fillna(0, inplace=True)

In [ ]:
test.shape

In [ ]:
preds=stacking_regressor.predict(test.drop('id', axis=1))
preds

In [ ]:
submission=pd.DataFrame({'id':test_logs.id.unique(), 'score':preds})
submission.to_csv('submission.csv', index=False)